# "Ranking"

- toc: false
- comments: true
- hide: false
- search_exclude: true

So we have some kind of reading club at our company where we normally share papers that we find interesting, so one of us has to explain the paper to the rest. 
This is week was my turn and since we had lately been talking about recommendation I thought on looking for a paper on Ranking. But instead of finding a SOA paper,
I thought it might be interesting to go into the roots and start simple since I hadnt touched much about this topic before. I finally decided to talk about the 
paper by Microsoft named "" wchich is not precisely new. Once I started reading it some references to previous work pop out as expected but there was a common factor in 
all of them, a more basic concept named Ordinal Regression. So I decided to read about this and explain this concept before commenting on the paper. So I am gonna 
recreate my talk here starting from Ordinal Regression and touching later on Ranking as explained in the Microsoft paper. 